# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("city_weather_date.csv")
weather_data.dropna()

,city,temp,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,bubaque,22.59,69,GW,1585037014,55,11.28,-15.83,22.59,7.11
1,klaksvik,4.00,100,FO,1585037014,86,62.23,-6.59,4.00,6.20
2,busselton,30.73,51,AU,1585037014,13,-33.65,115.33,31.11,3.35
3,dikson,-10.41,69,RU,1585037014,89,73.51,80.55,-10.41,2.60
4,lebu,14.39,100,CL,1585037014,98,-37.62,-73.65,14.39,2.11
...,...,...,...,...,...,...,...,...,...,...
560,victoria point,21.83,75,AU,1585037040,83,-27.58,153.30,23.00,1.50
561,kang,28.59,0,BW,1585037040,36,-23.68,22.79,28.59,3.81
562,carutapera,27.99,51,BR,1585037040,75,-1.20,-46.02,27.99,3.26
563,daru,24.69,100,PG,1585037040,93,-9.08,143.21,24.69,1.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
Narrow down the DataFrame to find your ideal weather condition. 

For example:
A max temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.
Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.
* Drop any rows will null values.

In [5]:
narrowed_city_df = weather_data[(weather_data['Max Temp'] < 40) & 
                      (weather_data['Max Temp'] > 30) &
                      (weather_data['Cloudiness'] == 0) & 
                      (weather_data['Wind Speed'] < 10) ]
narrowed_city_df

,city,temp,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
23,karasburg,32.09,0,NaN,1585037015,18,-28.02,18.75,32.09,3.04
33,karratha,30.29,0,AU,1585037016,72,-20.74,116.85,30.29,6.83
63,birao,36.29,0,CF,1585037017,7,10.28,22.79,36.29,7.68
223,lashio,30.29,0,MM,1585037025,15,22.93,97.75,30.29,2.89
266,umm lajj,30.29,0,SA,1585037027,15,25.02,37.27,30.29,4.72
436,khatra,35.49,0,IN,1585037034,17,22.98,86.85,35.49,4.22
461,asosa,37.09,0,ET,1585037036,8,10.07,34.53,37.09,2.10
525,yamethin,35.29,0,MM,1585037039,14,20.43,96.15,35.29,6.66
526,ratnagiri,34.19,0,IN,1585037039,44,16.98,73.30,34.19,5.39


In [6]:
hotel_data = []

In [7]:
target_search = "hotel"
radius = 5000 

params = {
    "radius": radius,
    "keyword": target_search,
    "key": g_key
}

for index, row in narrowed_city_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    city = row["city"]
    country = row["Country"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_data.append({"name":name_address["results"][0]["name"], 
                         "city": city,
                         "country": country})

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [10]:
hotel_df = pd.DataFrame(hotel_data)
hotel_df = hotel_df.rename(columns={"name":"Hotel Name",
                                    "city":"City",
                                    "country":"Country"})
hotel_df 

,Hotel Name,City,Country
0,Sunset Chalets,karasburg,NaN
1,ibis Styles Karratha,karratha,AU
2,Mansu Hotel,lashio,MM
3,عروس الشاطئ للوحدات السكنية المفروشة,umm lajj,SA
4,Oishi Plaza,khatra,IN
5,Bamboo Paradise Hotel,asosa,ET
6,yay tu ma hotel,yamethin,MM
7,Hotel Sea Fans,ratnagiri,IN


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [13]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=hotel_df, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1,
                                 info_box_content = hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(heat_layer)

# Display Map
fig

NameError: name 'hotel_info' is not defined